In [11]:
import pandas as pd
import numpy as np

In [24]:
# Read file
video_data = pd.read_csv('videos-stats.csv', usecols = [5,6,7], encoding='latin-1')
# Get rows m, cols n
m,n = video_data.shape
print(m)
print(n)

1846
3


In [36]:
# Def: Standardize the matrix
def Y_ij(data1):   
    for i in data1.columns:
       for j in range(n+1):
           if i == str(f'X{j}negative'):  # negative
               data1[i]=(np.max(data1[i])-data1[i])/(np.max(data1[i])-np.min(data1[i]))
           else:   # positive
               data1[i]=(data1[i]-np.min(data1[i]))/(np.max(data1[i])-np.min(data1[i]))
    return data1

In [37]:
# Def: Caculate the entropy
# Build empty matrix
None_ij = [[None] * n for i in range(m)]
def E_j(data2):
    data2 = np.array(data2)
    E = np.array(None_ij)
    for i in range(m):
        for j in range(n):
            if data2[i][j] == 0:
                e_ij = 0.0
            else:
                p_ij = data2[i][j] / data2.sum(axis=0)[j]  
                e_ij = (-1 / np.log(m)) * p_ij * np.log(p_ij)
            E[i][j] = e_ij
    E_j=E.sum(axis=0)
    return E_j

In [39]:
# Standardize the matrix
Y_ij = Y_ij(video_data)

In [40]:
# Get the entropy value
E_j = E_j(Y_ij)

In [41]:
# Calculate the information utility value
D_j = 1 - E_j

In [42]:
# Calculate the weight
W_j = D_j / sum(D_j)

In [43]:
# Get the weight
IW = pd.Series(W_j, index=video_data.columns, name='Index Weight')
print(IW)

Likes       0.301295
Comments    0.294011
Views       0.404695
Name: Index Weight, dtype: object


In [44]:
# Calculate the weighted matrix
Y_ij = np.array(Y_ij)  # Standard matrix Y_ij
Z_ij = np.array(None_ij)  # Empty matrix N_ij
for i in range(m):
    for j in range(n):
        Z_ij[i][j]=Y_ij[i][j]*W_j[j]  


In [54]:
# Calculate the optimum and the worst
Imax_j = Z_ij.max(axis=0)  
Imin_j = Z_ij.min(axis=0)  
Dmax_ij = np.array(None_ij)
Dmin_ij = np.array(None_ij)
for i in range(m):
    for j in range(n):
        Dmax_ij[i][j] = (Imax_j[j] - Z_ij[i][j]) ** 2
        Dmin_ij[i][j] = (Imin_j[j] - Z_ij[i][j]) ** 2

In [63]:
# Calculate the pop score
Dmax_i = Dmax_ij.sum(axis=1)**0.5  
Dmin_i = Dmin_ij.sum(axis=1)**0.5  
S_i = Dmin_i/(Dmin_i + Dmax_i)
S_i= pd.Series(S_i, index=video_data.index, name='pop_score')

In [64]:
# Append new column
data = pd.read_csv('videos-stats.csv', encoding='latin-1')
data['pop_score'] = S_i
data.to_csv('videos-stats.csv', index = False, sep = ',')
data = pd.read_csv('videos-stats.csv')